In [22]:
# Import your predictor
from linear_slope_predictor import LinearSlopePredictor
from window_peak_average import window_peak_val
# Example: from PREDICTOR_FILE import PREDICTOR_CLASS

# Add your predictor to the dictionary
predictors = {
    "LinearSlopePredictor": LinearSlopePredictor(),
    "window_peak_val": window_peak_val(),
    # Example: "PREDICTOR_CLASS": PREDICTOR_CLASS_CONSTRUCTOR_CALL
}

In [26]:
# Get raw data
import pandas as pd
from window_producer import raw_to_labeled_windows

raw_data = None
# If this fails, run get_raw_data.py 
raw_data = pd.read_csv("../combined_data.csv")

In [24]:
# Filter raw data

# raw_data = raw_data[raw_data['class'] != 0]
raw_data.loc[raw_data['class'] != 2] = 0
raw_data.loc[raw_data['class'] == 2] = 1
raw_data = raw_data.rename(columns={
    "class": "y",
    "channel4": "x"
})

In [27]:
# Label subset of raw data for train/test
train_data = raw_data.head(1000000)
x_windows_train, y_labels_train = raw_to_labeled_windows(train_data, min_window_size=200, padding=0)

test_data = raw_data.tail(1000000)
x_labels_test, y_labels_test = raw_to_labeled_windows(test_data, min_window_size=200, padding=0)

TypeError: raw_to_labeled_windows() got an unexpected keyword argument 'min_window_size'

In [ ]:
# print(x_windows_train)
# print(y_labels_train)

# print(x_labels_test)
# print(y_labels_test)

In [ ]:
# Train all predictors
for p in predictors:
    predictors[p].train(x_windows_train, y_labels_train)

In [ ]:
# Test all predictors
predictor_results = {}
for p in predictors:
    predictor_results[p] = {
        'tp': 0,
        'fp': 0,
        'tn': 0,
        'fn': 0,
    }
    for i in range(len(x_labels_test)):
        pred = predictors[p].label_window(x_labels_test[i])
        actual = y_labels_test[i]

        if pred and actual:
            predictor_results[p]['tp'] += 1
        elif pred and not actual:
            predictor_results[p]['fp'] += 1
        elif not pred and not actual:
            predictor_results[p]['tn'] += 1
        elif not pred and actual:
            predictor_results[p]['fn'] += 1
print(predictor_results)

{'LinearSlopePredictor': {'tp': 0, 'fp': 0, 'tn': 5, 'fn': 4}}
